# Customer Churn Prediction - EDA and Feature Engineering
## Industry-Standard Data Analysis Pipeline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load and Inspect Data

In [ ]:
df = pd.read_csv('../data/telco_churn.csv')
print(f"Dataset Shape: {df.shape}")
print(f"\nMemory Usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## 2. Data Quality Assessment

In [ ]:
# Missing values
missing = df.isnull().sum()
missing_pct = 100 * missing / len(df)
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Percentage', ascending=False))

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

# Check unique values for categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"\nCategorical columns: {len(categorical_cols)}")
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")

## 3. Target Variable Analysis

In [ ]:
# Churn distribution
churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(churn_counts.index, churn_counts.values, color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Churn Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn')
axes[0].set_ylabel('Count')
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

axes[1].pie(churn_pct.values, labels=churn_pct.index, autopct='%1.1f%%',
            colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[1].set_title('Churn Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Churn Rate: {churn_pct['Yes']:.2f}%")
print(f"Class Imbalance Ratio: {churn_counts['No'] / churn_counts['Yes']:.2f}:1")

## 4. Numerical Features Analysis

In [ ]:
# Convert TotalCharges to numeric (handle spaces)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill missing TotalCharges (new customers with 0 tenure)
df['TotalCharges'].fillna(df['MonthlyCharges'], inplace=True)

numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numerical_cols].describe()

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Numerical Features Distribution', fontsize=16, fontweight='bold')

for idx, col in enumerate(numerical_cols):
    # Histogram
    axes[0, idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    axes[0, idx].set_title(f'{col} Distribution')
    axes[0, idx].set_xlabel(col)
    axes[0, idx].set_ylabel('Frequency')
    
    # Box plot by Churn
    df.boxplot(column=col, by='Churn', ax=axes[1, idx])
    axes[1, idx].set_title(f'{col} by Churn Status')
    axes[1, idx].set_xlabel('Churn')
    axes[1, idx].set_ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
correlation = df[numerical_cols + ['SeniorCitizen']].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Numerical Features', fontsize=14, fontweight='bold')
plt.show()

## 5. Categorical Features Analysis

In [ ]:
# Exclude customerID and Churn from categorical analysis
cat_features = [col for col in categorical_cols if col not in ['customerID', 'Churn']]

# Plot churn rate by categorical features
n_cols = 3
n_rows = int(np.ceil(len(cat_features) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
axes = axes.flatten()

for idx, col in enumerate(cat_features):
    churn_by_cat = df.groupby(col)['Churn'].value_counts(normalize=True).unstack()
    churn_by_cat.plot(kind='bar', ax=axes[idx], color=['#2ecc71', '#e74c3c'])
    axes[idx].set_title(f'Churn Rate by {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Proportion')
    axes[idx].legend(['No Churn', 'Churn'])
    axes[idx].tick_params(axis='x', rotation=45)

# Hide extra subplots
for idx in range(len(cat_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 6. Feature Engineering

In [ ]:
# Create a copy for feature engineering
df_fe = df.copy()

# 1. Tenure grouping
df_fe['tenure_group'] = pd.cut(df_fe['tenure'], 
                                bins=[0, 12, 24, 48, 72], 
                                labels=['0-1 year', '1-2 years', '2-4 years', '4+ years'])

# 2. Average monthly charges
df_fe['AvgMonthlyCharges'] = df_fe['TotalCharges'] / (df_fe['tenure'] + 1)

# 3. Charge increase indicator
df_fe['ChargeIncrease'] = (df_fe['MonthlyCharges'] > df_fe['AvgMonthlyCharges']).astype(int)

# 4. Service combinations
service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 
                'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                'TechSupport', 'StreamingTV', 'StreamingMovies']

df_fe['TotalServices'] = (df_fe[service_cols] == 'Yes').sum(axis=1)

# 5. Contract and payment interaction
df_fe['Contract_Payment'] = df_fe['Contract'] + '_' + df_fe['PaymentMethod']

# 6. Senior citizen with partner
df_fe['SeniorWithPartner'] = ((df_fe['SeniorCitizen'] == 1) & 
                               (df_fe['Partner'] == 'Yes')).astype(int)

# 7. Has any add-on service
addon_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport']
df_fe['HasAddonService'] = (df_fe[addon_services] == 'Yes').any(axis=1).astype(int)

# 8. Has streaming service
df_fe['HasStreamingService'] = ((df_fe['StreamingTV'] == 'Yes') | 
                                 (df_fe['StreamingMovies'] == 'Yes')).astype(int)

print("New Features Created:")
print(df_fe[['tenure_group', 'AvgMonthlyCharges', 'ChargeIncrease', 
             'TotalServices', 'HasAddonService', 'HasStreamingService']].head())

In [ ]:
# Analyze new features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total Services
df_fe.groupby('TotalServices')['Churn'].value_counts(normalize=True).unstack().plot(
    kind='bar', ax=axes[0, 0], color=['#2ecc71', '#e74c3c'])
axes[0, 0].set_title('Churn Rate by Total Services', fontweight='bold')
axes[0, 0].set_xlabel('Total Services')
axes[0, 0].legend(['No Churn', 'Churn'])

# Tenure Group
df_fe.groupby('tenure_group')['Churn'].value_counts(normalize=True).unstack().plot(
    kind='bar', ax=axes[0, 1], color=['#2ecc71', '#e74c3c'])
axes[0, 1].set_title('Churn Rate by Tenure Group', fontweight='bold')
axes[0, 1].set_xlabel('Tenure Group')
axes[0, 1].legend(['No Churn', 'Churn'])

# Charge Increase
df_fe.groupby('ChargeIncrease')['Churn'].value_counts(normalize=True).unstack().plot(
    kind='bar', ax=axes[1, 0], color=['#2ecc71', '#e74c3c'])
axes[1, 0].set_title('Churn Rate by Charge Increase', fontweight='bold')
axes[1, 0].set_xlabel('Charge Increase (0=No, 1=Yes)')
axes[1, 0].legend(['No Churn', 'Churn'])

# Has Addon Service
df_fe.groupby('HasAddonService')['Churn'].value_counts(normalize=True).unstack().plot(
    kind='bar', ax=axes[1, 1], color=['#2ecc71', '#e74c3c'])
axes[1, 1].set_title('Churn Rate by Add-on Services', fontweight='bold')
axes[1, 1].set_xlabel('Has Add-on Service (0=No, 1=Yes)')
axes[1, 1].legend(['No Churn', 'Churn'])

plt.tight_layout()
plt.show()

## 7. Save Processed Data

In [ ]:
# Save the feature-engineered dataset
df_fe.to_csv('../data/telco_churn_processed.csv', index=False)
print(f"Processed data saved with shape: {df_fe.shape}")
print(f"New columns added: {set(df_fe.columns) - set(df.columns)}")

## 8. Key Insights Summary

### Findings:
1. **Class Imbalance**: Churn rate is approximately 26-27%, indicating moderate imbalance
2. **Tenure**: Strong inverse relationship with churn - longer tenure reduces churn probability
3. **Contract Type**: Month-to-month contracts show highest churn rates
4. **Services**: Customers with fewer services are more likely to churn
5. **Payment Method**: Electronic check users show higher churn rates
6. **Internet Service**: Fiber optic customers have higher churn compared to DSL
7. **Add-on Services**: Lack of security, backup, and support services correlates with higher churn

### Recommendations for Modeling:
- Use SMOTE or class weights to handle imbalance
- Focus on contract type and tenure as strong predictors
- Consider ensemble methods for better performance
- Feature interactions are important (Contract + Payment, Services combination)